In [1]:
import xgboost as xgb
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

In [16]:
full_results = pd.read_csv('full_results.csv')

full_stats = pd.read_csv('FullKenPom_pt.csv')

#full_results
print(full_stats.columns)

Index(['Unnamed: 0', 'Season', 'TeamName', 'Tempo', 'AdjTempo', 'OE', 'AdjOE',
       'DE', 'AdjDE', 'AdjEM', 'seed', 'ConfTournament', 'SOSAdjEM',
       'NCSOSAsjEM', 'O_eFG_Pct', 'D_eFG_Pct', 'O-D_eFG_Pct', 'O_TO_Pct',
       'D_TO_Pct', 'D-O_TO_Pct', 'O_OR_Pct', 'D_OR_Pct', 'O-D_OR_Pct',
       'O_FT_Rate', 'D_FT_Rate', 'O-D_FT_Rate', 'LastTenRecord'],
      dtype='object')


In [65]:
full_stats

stats_vec = ["Season",
             "TeamName",
             "AdjTempo", 
            "AdjOE",
            "AdjDE",
            "AdjEM",
            "seed",
            "ConfTournament",
            "SOSAdjEM",
            "NCSOSAsjEM",
            "O-D_eFG_Pct",
            "D-O_TO_Pct",
            "O-D_OR_Pct",
            "O-D_FT_Rate",
            "LastTenRecord"]

print(len(stats_vec))

target_stats = pd.DataFrame(columns = stats_vec)

for i in range(len(stats_vec)):
    target_stats[stats_vec[i]] = full_stats[stats_vec[i]]

target_stats.columns

target_stats.head()

15


,Season,TeamName,AdjTempo,AdjOE,AdjDE,AdjEM,seed,ConfTournament,SOSAdjEM,NCSOSAsjEM,O-D_eFG_Pct,D-O_TO_Pct,O-D_OR_Pct,O-D_FT_Rate,LastTenRecord
0,2002,Duke,73.9016,121.374,87.3522,34.0220,1,1,9.87,6.66,10.2329,7.1166,0.3987,8.8762,0.8
1,2002,Cincinnati,66.9485,117.448,86.6033,30.8442,1,1,6.58,3.48,10.1412,4.1339,6.3908,10.1433,0.9
2,2002,Kansas,77.6019,118.100,90.3113,27.7883,1,0,10.67,8.32,8.6710,0.2172,8.6870,5.8071,0.9
3,2002,Maryland,73.3263,118.151,91.2974,26.8534,1,0,9.88,1.62,8.1459,2.1846,2.4983,10.6008,0.9
4,2002,Florida,69.5612,115.387,89.6694,25.7176,5,0,9.11,-0.56,6.7968,4.6986,5.3645,1.6048,0.6


In [132]:

###mask test and train data###
res_test_mask = (full_results['Year'] == 2016) | (full_results['Year'] == 2017) | (full_results['Year'] == 2018)
stats_test_mask = (full_stats['Season'] == 2016) | (full_stats['Season'] == 2017) | (full_stats['Season'] == 2018)
res_train_mask = (full_results['Year'] < 2016)
stats_train_mask = (full_stats['Season'] < 2016)

res_data_test = full_results[res_test_mask]
stats_data_test = full_stats[stats_test_mask]
res_data_train = full_results[res_train_mask]
stats_data_train = full_stats[stats_train_mask]

#reindex all of them
res_data_test = res_data_test.reset_index()
stats_data_test = stats_data_test.reset_index()
res_data_train = res_data_train.reset_index()
stats_data_train = stats_data_train.reset_index()

for i in range(len(res_data_test)):
    if res_data_test['Region Name'][i] == "First Four":
        res_data_test = res_data_test.drop(i)
for i in range(len(res_data_train)):
    if res_data_train['Region Name'][i] == "First Four":
        res_data_train = res_data_train.drop(i)
        
#reindex all of them
res_data_test = res_data_test.reset_index()
stats_data_test = stats_data_test.reset_index()
res_data_train = res_data_train.reset_index()
stats_data_train = stats_data_train.reset_index()

res_data_train.head()


,level_0,index,Unnamed: 0,Year,Round,Region Number,Region Name,SeedA,ScoreA,TeamA,TeamB,ScoreB,SeedB
0,1,1,1,2002,1,1,South,1,84,Duke,Winthrop,37,16
1,2,2,2,2002,1,1,South,2,86,Alabama,Florida Atlantic,78,15
2,3,3,3,2002,1,1,South,3,71,Pittsburgh,Central Connecticut St,54,14
3,4,4,4,2002,1,1,South,4,89,USC,UNC Wilmington,93,13
4,5,5,5,2002,1,1,South,5,75,Indiana,Utah,56,12


In [133]:
training_data = np.zeros((len(res_data_train), len(stats_vec)-2))
print(np.shape(training_data))

Season = res_data_train['Year'][1]
TeamA = res_data_train['TeamA'][1]
TeamB = res_data_train['TeamB'][1]
    
A_mask = (stats_data_train['Season'] == Season) & (stats_data_train['TeamName'] == TeamA)                        
A = stats_data_train[A_mask]
#print(A)
    
B_mask = (stats_data_train['Season'] == Season) & (stats_data_train['TeamName'] == TeamB)
B = stats_data_train[B_mask]

A_value = A[stats_vec[2]].iloc[0]
#A_value = A_series.iloc[0]
print(A_value)
#print(B)



for j in range(2,len(stats_vec)): 
    training_data[0,j-2] = A[stats_vec[j]].iloc[0] - B[stats_vec[j]].iloc[0]

training_data[0,:]

(882, 13)
67.8101


array([ -2.0491 ,  15.7256 ,  -7.3805 ,  23.10542, -13.     ,  -1.     ,
        13.53   ,   2.96   ,   6.0373 ,  -2.5248 ,   3.1176 ,  14.1589 ,
        -0.1    ])

In [135]:
training_data = np.zeros((len(res_data_train), len(stats_vec)))
print(np.shape(training_data))


for i in range(len(res_data_train)): 
    Season = res_data_train['Year'][i]
    TeamA = res_data_train['TeamA'][i]
    TeamB = res_data_train['TeamB'][i]
    
    A_mask = (stats_data_train['Season'] == Season) & (stats_data_train['TeamName'] == TeamA)                        
    A = stats_data_train[A_mask]
    #print(A)
    
    B_mask = (stats_data_train['Season'] == Season) & (stats_data_train['TeamName'] == TeamB)
    B = stats_data_train[B_mask]

    #A_value = A[stats_vec[2]].iloc[0]
    #A_value = A_series.iloc[0]
    #print(A_value)
    #print(B)



    for j in range(2,len(stats_vec)): 
        training_data[i,j-2] = A[stats_vec[j]].iloc[0] - B[stats_vec[j]].iloc[0]
    print(i)
    print(training_data)
  
        
                                 

(882, 15)
0
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
1
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
2
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.      

 [  0.       0.       0.     ...   0.       0.       0.    ]]
30
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
31
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
32
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 .

[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
77
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
78
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 

101
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
102
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
103
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0. 

131
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
132
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
133
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0. 

...

716
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
717
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
718
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0. 

745
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
746
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
747
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0. 

[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
792
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
793
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]

830
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
831
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]
 [  0.       0.       0.     ...   0.       0.       0.    ]]
832
[[  8.2874  24.2208 -14.8448 ...   0.       0.       0.    ]
 [ -2.0491  15.7256  -7.3805 ...  -0.1      0.       0.    ]
 [ -0.3923  11.692   -6.6079 ...  -0.1      0.       0.    ]
 ...
 [  0.       0.       0.     ...   0.       0.       0. 

IndexError: single positional indexer is out-of-bounds

In [8]:
###mask test data###
test_mask = (full_results['Year'] == 2016) | (full_results['Year'] == 2017) | (full_results['Year'] == 2018)
data_test = full_results[test_mask]

###getting rid of non-float columns (Region Name, TeamA, TeamB)###
data_clean_mask = (full_results.columns.values != 'Region Name') & (full_results.columns.values != 'TeamA') & (full_results.columns.values != 'TeamB')

data_test_clean = data_test.columns[data_clean_mask]
test_columns_list = list(data_test_clean)

new_data_test = pd.DataFrame(columns = test_columns_list)

for i in range(len(test_columns_list)):
    new_data_test[test_columns_list[i]] = data_test[test_columns_list[i]]
    

###mask train data###
train_mask = (full_results['Year'] != 2016) & (full_results['Year'] != 2017) & (full_results['Year'] != 2018)
data_train = full_results[train_mask]

###getting rid of non-float columns (Region Name, TeamA, TeamB)###
data_clean_mask = (full_results.columns.values != 'Region Name') & (full_results.columns.values != 'TeamA') & (full_results.columns.values != 'TeamB')

data_train_clean = data_train.columns[data_clean_mask]
train_columns_list = list(data_train_clean)

new_data_train = pd.DataFrame(columns = train_columns_list)

for i in range(len(train_columns_list)):
    new_data_train[train_columns_list[i]] = data_train[train_columns_list[i]]


#label = 

new_data_test

,Unnamed: 0,Year,Round,Region Number,SeedA,ScoreA,ScoreB,SeedB
911,911,2016,0,0,11,50,70,11
912,912,2016,0,0,16,55,59,16
913,913,2016,0,0,16,96,65,16
914,914,2016,0,0,11,67,62,11
915,915,2016,1,1,1,105,79,16
916,916,2016,1,1,2,86,56,15
917,917,2016,1,1,3,79,72,14
918,918,2016,1,1,4,66,77,13
919,919,2016,1,1,5,79,74,12
920,920,2016,1,1,6,55,65,11


In [58]:
#train = xgb.DMatrix(data_train, label=label)
test = xgb.DMatrix(new_data_test, label = label)

TypeError: must be real number, not str